In [1]:
import os
from math import prod

from typing import *

import pandas as pd
import numpy as np

import einops

import tensorly as tl
from tensorly.decomposition import parafac, non_negative_parafac
from tensorly.kruskal_tensor import kruskal_to_tensor

Using numpy backend.


In [2]:
pd.set_option('display.max_rows', 500)

In [3]:
df = pd.read_json(os.path.expanduser("/home/shibbiry/projects/dctn/small_experiments/logmatmulexp_benchmark/results.json"))
#df = df[~ (df["device"] == "cpu")] # look only at GPUs
#df = df[~ (df["dim_size"] == 64)] # small dim size execution times are too chaotic
df = df.set_index(["dim_size", "func", "device", "dtype"]).sort_index()

In [4]:
df

forward_backward_seconds_per_iteration  \
dim_size func                device           dtype                                                   
64       logmatmulexp        GeForce GTX 1070 torch.float32                                0.001806   
                                              torch.float64                                0.001583   
                             Graphics Device  torch.float32                                0.001700   
                                              torch.float64                                0.001644   
                             cpu              torch.float32                                0.004893   
                                              torch.float64                                0.010602   
         logmatmulexp_lowmem GeForce GTX 1070 torch.float32                                0.003096   
                                              torch.float64                                0.003193   
                             Graphics Device  torch.float32                                0.003049   
                                              torch.float64                                0.002803   
                             cpu              torch.float32                                0.009114   
                                              torch.float64                                0.016853   
         matmul              GeForce GTX 1070 torch.float32                                0.000492   
                                              torch.float64                                0.000463   
                             Graphics Device  torch.float32                                0.000376   
                                              torch.float64                                0.000311   
                             cpu              torch.float32                                0.000270   
                                              torch.float64                                0.000300   
128      logmatmulexp        GeForce GTX 1070 torch.float32                                0.003153   
                                              torch.float64                                0.007231   
                             Graphics Device  torch.float32                                0.001676   
                                              torch.float64                                0.004302   
                             cpu              torch.float32                                0.059916   
                                              torch.float64                                0.107174   
         logmatmulexp_lowmem GeForce GTX 1070 torch.float32                                0.004703   
                                              torch.float64                                0.010761   
                             Graphics Device  torch.float32                                0.003089   
                                              torch.float64                                0.006252   
                             cpu              torch.float32                                0.094523   
                                              torch.float64                                0.164866   
         matmul              GeForce GTX 1070 torch.float32                                0.000441   
                                              torch.float64                                0.000464   
                             Graphics Device  torch.float32                                0.000471   
                                              torch.float64                                0.000355   
                             cpu              torch.float32                                0.000511   
                                              torch.float64                                0.000703   
150      logmatmulexp        GeForce GTX 1070 torch.float32                                0.005013   
                                            

In [5]:
shape = df.index.levshape
assert prod(shape) == len(df)

In [6]:
what = ["forward_backward_seconds_per_iteration", "forward_seconds_per_iteration"]
#what = ["forward_seconds_per_iteration"]

In [7]:
array = df[what].values.reshape(shape[0], shape[1], shape[2], shape[3], len(what))
print(array.shape) # dim_size × func × device × dtype × what
print(df.index.levels)

(6, 3, 3, 2, 2)
[[64, 128, 150, 192, 256, 280], ['logmatmulexp', 'logmatmulexp_lowmem', 'matmul'], ['GeForce GTX 1070', 'Graphics Device', 'cpu'], ['torch.float32', 'torch.float64']]


In [8]:
factors = non_negative_parafac(array, rank=1, tol=1e-8, verbose=True)

reconstruction error=0.09083051780288953, variation=4.0929759581587177e-13.
converged in 2 iterations.


In [9]:
factors

[array([[0.02432675],
        [0.24575081],
        [0.43651074],
        [0.87500978],
        [2.18576621],
        [2.70063728]]),
 array([[0.54947385],
        [0.83549431],
        [0.00526884]]),
 array([[0.05550402],
        [0.03012612],
        [0.99800388]]),
 array([[0.50728041],
        [0.8617811 ]]),
 array([[0.91735388],
        [0.39807275]])]

In [10]:
def calc_errors(x: np.ndarray, factors) -> Tuple[np.ndarray, np.ndarray]:
    abs_err = np.abs(kruskal_to_tensor(factors) - x)
    rel_err = abs_err / np.abs(x)
    print(f"{np.max(abs_err)=:.1e}, {np.mean(abs_err)=:.1e}, {np.max(rel_err)=:.1e}, {np.mean(rel_err)=:.1e})")
    return abs_err, rel_err

In [11]:
abs_err, rel_err = calc_errors(array, factors)

np.max(abs_err)=1.7e-01, np.mean(abs_err)=7.8e-03, np.max(rel_err)=1.2e+00, np.mean(rel_err)=4.1e-01)


In [12]:
def array_to_df(x: np.ndarray) -> pd.DataFrame:
    return pd.DataFrame(x.reshape((-1, len(what))), index=df.index, columns=what)

In [13]:
array_to_df(rel_err)

forward_backward_seconds_per_iteration  \
dim_size func                device           dtype                                                   
64       logmatmulexp        GeForce GTX 1070 torch.float32                                0.808869   
                                              torch.float64                                0.629395   
                             Graphics Device  torch.float32                                0.889795   
                                              torch.float64                                0.806387   
                             cpu              torch.float32                                0.268678   
                                              torch.float64                                0.005256   
         logmatmulexp_lowmem GeForce GTX 1070 torch.float32                                0.830441   
                                              torch.float64                                0.720648   
                             Graphics Device  torch.float32                                0.906548   
                                              torch.float64                                0.827292   
                             cpu              torch.float32                                0.035660   
                                              torch.float64                                0.048489   
         matmul              GeForce GTX 1070 torch.float32                                0.993272   
                                              torch.float64                                0.987857   
                             Graphics Device  torch.float32                                0.995224   
                                              torch.float64                                0.990195   
                             cpu              torch.float32                                0.779747   
                                              torch.float64                                0.662456   
128      logmatmulexp        GeForce GTX 1070 torch.float32                                0.106315   
                                              torch.float64                                0.180643   
                             Graphics Device  torch.float32                                0.129213   
                                              torch.float64                                0.252451   
                             cpu              torch.float32                                0.046680   
                                              torch.float64                                0.005926   
         logmatmulexp_lowmem GeForce GTX 1070 torch.float32                                0.127625   
                                              torch.float64                                0.162763   
                             Graphics Device  torch.float32                                0.068285   
                                              torch.float64                                0.217901   
                             cpu              torch.float32                                0.008835   
                                              torch.float64                                0.017408   
         matmul              GeForce GTX 1070 torch.float32                                0.924200   
                                              torch.float64                                0.877640   
                             Graphics Device  torch.float32                                0.961470   
                                              torch.float64                                0.913221   
                             cpu              torch.float32                                0.176599   
                                              torch.float64                                0.452440   
150      logmatmulexp        GeForce GTX 1070 torch.float32                                0.235913   
                                            

In [14]:
# now let's normalize array before factoring it once more

In [15]:
normalized_array = array.copy()
for dim, factor in enumerate(factors):
    factor = einops.rearrange(factor, f"i () -> {' '.join('()' for i in range(dim))} i {' '.join('()' for i in range(array.ndim-dim-1))}")
    normalized_array /= factor

In [16]:
print(f"{array.std()=:.1e}, {normalized_array.std()=:.1e}")

array.std()=2.3e-01, normalized_array.std()=2.1e+01


In [17]:
normalized_factors = non_negative_parafac(normalized_array, 1, tol=1e-8, verbose=True)

reconstruction error=0.10227731704582028, variation=1.9082610491771845e-11.
converged in 2 iterations.


In [18]:
normalized_factors

[array([[314.93030605],
        [ 35.83708516],
        [ 18.41960939],
        [ 12.31981299],
        [  7.21467436],
        [  7.37381266]]),
 array([[0.05186907],
        [0.05307245],
        [0.99724282]]),
 array([[0.56375102],
        [0.82557455],
        [0.02475631]]),
 array([[0.88870711],
        [0.45847568]]),
 array([[0.91593733],
        [0.40132469]])]

In [19]:
normalized_abs_err, normalized_rel_err = calc_errors(normalized_array, normalized_factors)

np.max(abs_err)=1.7e+01, np.mean(abs_err)=1.4e+00, np.max(rel_err)=2.1e+00, np.mean(rel_err)=6.9e-01)


In [20]:
new_factors = [normalized_factor * old_factor for normalized_factor, old_factor in zip(normalized_factors, factors)]

In [21]:
new_abs_err, new_rel_err = calc_errors(array, new_factors)

np.max(abs_err)=1.8e+00, np.mean(abs_err)=8.0e-02, np.max(rel_err)=2.1e+00, np.mean(rel_err)=6.9e-01)


In [22]:
array_to_df(new_rel_err).loc[pd.IndexSlice[:, "matmul"], :]

forward_backward_seconds_per_iteration  \
dim_size func   device           dtype                                                   
64       matmul GeForce GTX 1070 torch.float32                                0.030365   
                                 torch.float64                                0.097155   
                Graphics Device  torch.float32                                0.008064   
                                 torch.float64                                0.067577   
                cpu              torch.float32                                0.393951   
                                 torch.float64                                0.102083   
128      matmul GeForce GTX 1070 torch.float32                                0.243126   
                                 torch.float64                                0.035246   
                Graphics Device  torch.float32                                0.074637   
                                 torch.float64                                0.075191   
                cpu              torch.float32                                0.152631   
                                 torch.float64                                0.460365   
150      matmul GeForce GTX 1070 torch.float32                                0.209209   
                                 torch.float64                                0.357766   
                Graphics Device  torch.float32                                0.226234   
                                 torch.float64                                0.098617   
                cpu              torch.float32                                0.614319   
                                 torch.float64                                0.800678   
192      matmul GeForce GTX 1070 torch.float32                                0.499501   
                                 torch.float64                                0.433409   
                Graphics Device  torch.float32                                0.099126   
                                 torch.float64                                0.039624   
                cpu              torch.float32                                0.909559   
                                 torch.float64                                0.909058   
256      matmul GeForce GTX 1070 torch.float32                                2.143685   
                                 torch.float64                                0.647956   
                Graphics Device  torch.float32                                0.806121   
                                 torch.float64                                0.479755   
                cpu              torch.float32                                0.959555   
                                 torch.float64                                0.929852   
280      matmul GeForce GTX 1070 torch.float32                                1.270004   
                                 torch.float64                                0.635420   
                Graphics Device  torch.float32                                0.909169   
                                 torch.float64                                0.462490   
                cpu              torch.float32                                0.706091   
                                 torch.float64                                0.849979   

                                                forward_seconds_per_iteration  
dim_size func   device           dtype                                         
64       matmul GeForce GTX 1070 torch.float32                       0.357750  
                                 torch.float64                       0.140836  
                Graphics Device  torch.float32                       0.096116  
                                 torch.float64                       0.061141  
                cpu              torch.float32                       0.399415  
                                 torch.float64                     

In [26]:
for level, factor in zip((*df.index.levels, what), new_factors):
    name = level.name if isinstance(level, pd.Int64Index) else "what"
    print(pd.Series(index=level, data=factor.flatten() / factor.min(), name=name))
    print()
    #print(tuple(zip(level, factor.flatten())))

dim_size
64     1.000000
128    1.149553
150    1.049486
192    1.407079
256    2.058363
280    2.599320
Name: dim_size, dtype: float64

func
logmatmulexp           5.424247
logmatmulexp_lowmem    8.439108
matmul                 1.000000
Name: what, dtype: float64

device
GeForce GTX 1070    1.266466
Graphics Device     1.006657
cpu                 1.000000
Name: what, dtype: float64

dtype
torch.float32    1.141021
torch.float64    1.000000
Name: what, dtype: float64

forward_backward_seconds_per_iteration    5.259498
forward_seconds_per_iteration             1.000000
Name: what, dtype: float64



In [42]:
(df.loc[pd.IndexSlice[:, :, "cpu"], :][what].droplevel(2) / df.loc[pd.IndexSlice[:, :, "Graphics Device"], :][what].droplevel(2)).mean()

forward_backward_seconds_per_iteration    23.326752
forward_seconds_per_iteration             27.936942
dtype: float64

# All-fucking-right, I'll do this shitty factor analysis with my own hands

In [75]:
def extract_factor(array: np.ndarray, dim: int, baseline_slice_index: int) -> np.ndarray:
    baseline_slice = array[tuple(slice(None) for i in range(dim)) + (baseline_slice_index,)]
    elems = []
    for i in range(array.shape[dim]):
        this_slice = array[tuple(slice(None) for i in range(dim)) + (i,)]
        elems.append((this_slice / baseline_slice).mean())
    return np.array(elems)

In [76]:
def print_factor(factor: np.ndarray, dim: int) -> pd.Series:
    if dim < len(df.index.levels):
        index = df.index.levels[dim]
    else:
        index = what
    print(pd.Series(index=index, data=factor))

In [80]:
for dim, baseline_slice_index in (
    (0, 3),
    (1, 0),
    (2, 1),
    (3, 0),
    (4, 1)
):
    print_factor(extract_factor(array, dim, baseline_slice_index), dim)
    print()

dim_size
64     0.242170
128    0.388777
150    0.532644
192    1.000000
256    2.154617
280    2.615546
dtype: float64

func
logmatmulexp           1.000000
logmatmulexp_lowmem    1.298112
matmul                 0.054123
dtype: float64

device
GeForce GTX 1070     1.638247
Graphics Device      1.000000
cpu                 25.631847
dtype: float64

dtype
torch.float32    1.000000
torch.float64    2.467929
dtype: float64

forward_backward_seconds_per_iteration    2.83085
forward_seconds_per_iteration             1.00000
dtype: float64

